# CALIPSO HDF to Tiled COPC Conversion Pipeline

Complete pipeline: **HDF4 → LAS 1.4 → 4 Latitude-Tiled COPC Files**

## Why Latitude Tiling?

PDAL's COPC writer creates corrupted cube bounds for globe-spanning data:
- ❌ Problem: Latitude values like 304° (invalid)
- ✅ Solution: Split into 4 latitude tiles before COPC conversion
- 🚀 Benefits: Valid bounds + root node intersection + 75% tile skipping

## Latitude Tiles

| Tile | Latitude Range |
|------|----------------|
| south | -90° to -30° |
| south_mid | -30° to 0° |
| north_mid | 0° to 30° |
| north | 30° to 90° |

## Directory Structure

```
data/
├── raw/              # HDF files
├── converted_las/    # Intermediate LAS
└── final/
    └── tiled_copc/   # Final tiled COPC (4 per timestamp)
```

## 1. Setup

In [ ]:
import sys
import subprocess
import json
from pathlib import Path

# Add parent directory to import calipso_to_las
sys.path.insert(0, str(Path.cwd().parent))
from calipso_to_las import convert_calipso_to_las

# Configure paths
DATA_DIR = Path('.')
RAW_DIR = DATA_DIR / 'raw'
LAS_DIR = DATA_DIR / 'converted_las'
TILED_COPC_DIR = DATA_DIR / 'final' / 'tiled_copc'

# Create directories
LAS_DIR.mkdir(exist_ok=True, parents=True)
TILED_COPC_DIR.mkdir(exist_ok=True, parents=True)

PDAL_BIN = '/opt/anaconda3/envs/pdal/bin/pdal'

print(f'Output directory: {TILED_COPC_DIR}')

In [ ]:
import glob

hdf_files = sorted(glob.glob(str(RAW_DIR / '*.hdf')))
print(f'Found {len(hdf_files)} HDF files:\n')
for i, f in enumerate(hdf_files):
    size_mb = Path(f).stat().st_size / (1024**2)
    print(f'[{i}] {Path(f).name} ({size_mb:.1f} MB)')

In [ ]:
# SELECT FILE INDEX
FILE_INDEX = 0

hdf_path = Path(hdf_files[FILE_INDEX])
hdf_filename = hdf_path.stem
las_path = LAS_DIR / f'{hdf_filename}.las'

print(f'Selected: {hdf_path.name}')
print(f'LAS output: {las_path}')
print(f'COPC output dir: {TILED_COPC_DIR}')

## 2. HDF → LAS Conversion

In [ ]:
print('='*80)
print('STEP 1: HDF → LAS')
print('='*80)

convert_calipso_to_las(hdf_path, las_path)

las_size_mb = las_path.stat().st_size / (1024**2)
print(f'\n✅ Created {las_path.name} ({las_size_mb:.1f} MB)')

## 3. LAS → Tiled COPC Conversion

In [ ]:
tiles = [
    {'name': 'south', 'lat_min': -90, 'lat_max': -30},
    {'name': 'south_mid', 'lat_min': -30, 'lat_max': 0},
    {'name': 'north_mid', 'lat_min': 0, 'lat_max': 30},
    {'name': 'north', 'lat_min': 30, 'lat_max': 90}
]

print('='*80)
print('STEP 2: LAS → 4 Tiled COPC Files')
print('='*80)
print(f'Input: {las_path.name}')
print(f'Creating {len(tiles)} latitude tiles\n')

tile_results = {}

for i, tile in enumerate(tiles, 1):
    name = tile['name']
    lat_min, lat_max = tile['lat_min'], tile['lat_max']
    output = TILED_COPC_DIR / f'{hdf_filename}_tile_{name}.copc.laz'
    
    print(f'\n[{i}/{len(tiles)}] {name} (lat {lat_min}° to {lat_max}°)')
    
    pipeline = {
        'pipeline': [
            {'type': 'readers.las', 'filename': str(las_path)},
            {'type': 'filters.range', 'limits': f'Y[{lat_min}:{lat_max}]'},
            {'type': 'filters.stats', 'dimensions': 'X,Y,Z,Intensity'},
            {
                'type': 'writers.copc',
                'filename': str(output),
                'forward': 'all',
                'a_srs': 'EPSG:4326',
                'scale_x': 0.0001,
                'scale_y': 0.0001,
                'scale_z': 0.001,
                'offset_x': 'auto',
                'offset_y': 'auto',
                'offset_z': 'auto'
            }
        ]
    }
    
    try:
        result = subprocess.run(
            [PDAL_BIN, 'pipeline', '--stdin'],
            input=json.dumps(pipeline).encode(),
            capture_output=True,
            check=True
        )
        size_mb = output.stat().st_size / (1024**2)
        print(f'  ✅ {output.name} ({size_mb:.1f} MB)')
        tile_results[name] = {'success': True, 'size_mb': size_mb}
    except Exception as e:
        print(f'  ❌ Error: {e}')
        tile_results[name] = {'success': False}

print(f'\n{'='*80}')
print(f'✅ Tiled COPC conversion complete!')
print(f'{'='*80}')

## 4. Summary

In [ ]:
total_size = sum(r['size_mb'] for r in tile_results.values() if r['success'])
successful = sum(1 for r in tile_results.values() if r['success'])

print(f'Input HDF: {hdf_path.name}')
print(f'Successful tiles: {successful}/{len(tiles)}')
print(f'Total tiled COPC size: {total_size:.1f} MB')
print(f'Original LAS size: {las_size_mb:.1f} MB')
print(f'Compression: {(1-total_size/las_size_mb)*100:.1f}% reduction')
print(f'\nOutput directory: {TILED_COPC_DIR}')
print(f'\nNext steps:')
print(f'  1. Change FILE_INDEX to process other files')
print(f'  2. Or use batch script for all files')
print(f'  3. Copy to web server: public/potree_data/tiled/')
print(f'  4. Update fileSearch.ts with new basenames')